# Query Operations

Objectives:
 * Learn how to use the query method for tabular datasets

In [ ]:
import h5pyd
# Open a file containing stock quote data
f = h5pyd.File("/home/john/snp500.h5", 'r')

In [ ]:
dset = f["dset"]
dset.shape

In [ ]:
# an example of compound type
dset.dtype

In [ ]:
# The date field starting in 1970
arr = dset[:10]  # get first 10 elements

In [ ]:
arr['date']  # date starts in 1970

In [ ]:
arr = dset[-10:]  # get last 10 elements

In [ ]:
arr['date']  # and ends in 2015

In [ ]:
# If we wanted to extract all stock quotes with the symbol AAPL
# We could read the dataset in chunks and filter out anything 
# other than that symbol, but it would be rather slow
#
# More efficient is to use the dset query operator which can just 
# return rows matching the specification
%time arr = dset.read_where("symbol == b'AAPL'")

In [ ]:
arr.shape

In [ ]:
arr[:5]

Problem: Try with another snp500 symbol